## Collaborative Filtering
Collaborative filtering is a machine learning technique that predicts ratings awarded to items by users.

### Import the ALS class
In this exercise, you will use the Alternating Least Squares collaborative filtering algorithm to creater a recommender.

In [1]:
from pyspark.ml.recommendation import ALS

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1577379476625_0013,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### Load Source Data
The source data for the recommender is in two files - one containing numeric IDs for movies and users, along with user ratings; and the other containing details of the movies.

In [23]:
ratings = spark.read.csv('wasb:///data/ratings.csv', inferSchema=True, header=True)
#movies = spark.read.csv('wasb:///data/moviess.csv', inferSchema=True, header=True)
#ratings.join(movies, "movieId").show(10)
ratings.show(10)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
+------+-------+------+----------+
only showing top 10 rows

### Prepare the Data
To prepare the data, split it into a training set and a test set.

In [13]:
data = ratings.select("userId", "movieId", "rating")
splits = data.randomSplit([0.7, 0.3])
train = splits[0].withColumnRenamed("rating", "label")
test = splits[1].withColumnRenamed("rating", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print "Training Rows:", train_rows, " Testing Rows:", test_rows

Training Rows: 70064  Testing Rows: 29940

### Build the Recommender
The ALS class is an estimator, so you can use its **fit** method to traing a model, or you can include it in a pipeline. Rather than specifying a feature vector and as label, the ALS algorithm requries a numeric user ID, item ID, and rating.

In [18]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="label")
model = als.fit(train)

### Test the Recommender
Now that you've trained the recommender, you can see how accurately it predicts known ratings in the test set.

In [22]:
prediction = model.transform(test)
prediction.select("userId", "movieId", "prediction", "trueLabel").show(10, truncate=False)

+------+-------+----------+---------+
|userId|movieId|prediction|trueLabel|
+------+-------+----------+---------+
|311   |463    |3.1048281 |3.0      |
|19    |471    |4.1637497 |3.0      |
|358   |471    |3.9488633 |5.0      |
|514   |471    |4.004389  |4.0      |
|399   |471    |3.670352  |5.0      |
|126   |471    |3.2524004 |5.0      |
|548   |471    |3.9809384 |4.0      |
|588   |471    |5.4306173 |3.0      |
|105   |471    |3.78558   |4.0      |
|299   |471    |3.6943057 |4.5      |
+------+-------+----------+---------+
only showing top 10 rows

The data used in this exercise describes 5-star rating activity from [MovieLens](http://movielens.org), a movie recommendation service. It was created by GroupLens, a research group in the Department of Computer Science and Engineering at the University of Minnesota, and is used here with permission.

This dataset and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/>.

For more information, see F. Maxwell Harper and Joseph A. Konstan. 2015. [The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015)](http://dx.doi.org/10.1145/2827872)